In [2]:
import requests as rq
from urllib.parse import urljoin
import pandas as pd
import tqdm
from urllib.parse import quote
from bs4 import BeautifulSoup


In [3]:
URL = "https://www.filmweb.pl"

In [4]:
class FilmwebClient:
    def __init__(self, server) -> None:
        self.server = server
        self.common_headers = {'x-locale': 'en'}
        self.users = []
        self.critics = []
        self.movies = []
        self.proxy = {
            'http': '162.55.95.91:2289',
            'http': '162.55.95.91:3857',
            'http': '217.182.170.224:80'
        }


    def search_movies(self, page: int):
        url = urljoin(self.server, f'/api/v1/films/search?page={page}')
        return rq.get(url, headers=self.common_headers)


    def get_critics(self):
        url = urljoin(self.server, '/api/v1/film/critics')
        return rq.get(url, headers=self.common_headers)
    

    def get_movie_rating(self, movie_id):
        url = urljoin(self.server, f'/api/v1/film/{movie_id}/rating')
        return rq.get(url, headers=self.common_headers)
    

    def get_movie_critics(self, movie_id):
        url = urljoin(self.server, f'/api/v1/film/{movie_id}/critics')
        return rq.get(url, headers=self.common_headers)
    

    def get_movie_review(self, movie_id, critic_username):
        url = urljoin(self.server, f'/api/v1/user/{critic_username}/vote/film/{movie_id}')
        return rq.get(url, headers=self.common_headers)


    def get_reviewed_movies(self, critic_username):
        url = urljoin(self.server, f'/api/v1/user/{critic_username}/vote/film/')
        return rq.get(url, headers=self.common_headers)
    

    def get_reviewed_servies(self, critic_username):
        url = urljoin(self.server, f'/api/v1/user/{critic_username}/vote/serial')
        return rq.get(url, headers=self.common_headers)
    
    
    def get_tv_series_review(self, series_id, critic_username):
        url = urljoin(self.server, f'/api/v1/user/{critic_username}/vote/serial/{series_id}')
        return rq.get(url, headers=self.common_headers)
    

    def get_title(self, id):
        url = urljoin(self.server, f'/api/v1/title/{id}/info')
        return rq.get(url, headers=self.common_headers, timeout=(3, 5), proxies=self.proxy)
    
    
    def get_movie_info(self, title):
        url = urljoin(self.server, f'/api/v1/film/{title}/info')
        return rq.get(url, headers=self.common_headers, timeout=(3, 5), proxies=self.proxy)
    
    def get_movie_discussion(self, movie_path, page=1):
        url = urljoin(self.server, f'/film/{movie_path}/discussion?plusMinus=true&page={page}')
        return rq.get(url, headers=self.common_headers, timeout=(3, 5), proxies=self.proxy)



In [5]:
filmweb = FilmwebClient(URL)

# Get movies

In [ ]:
def generator():
  while True:
    yield


page = 1
movies = []
for _ in tqdm.tqdm(generator()):
    response = filmweb.search_movies(page)
    if response.status_code != 200:
        print(f'Reached the end of movies list, last page: {page-1}, number of movies: {len(movies)}')
        break
    try:
        movies += response.json()['searchHits']
    except:
        print(f'Cannot parse movies from page {page}.')
    page+=1

Don't worry about above problem as long as the number of pages is reasonable.

In [8]:
df_movies = pd.DataFrame(movies)
df_movies.head()

,id,type
0,862,film
1,1048,film
2,998,film
3,671,film
4,9136,film


In [10]:
df_movies.count()

id      4630
type    4630
dtype: int64

Save so far collected data, just in case

In [69]:
df_movies[['id', 'type']].to_csv('intermidate/movies.csv', index=False)

# Get critics

In [14]:
df_critics = pd.DataFrame(filmweb.get_critics().json())
df_critics.head()

,id,userName,priority,publisher
0,1164541,michaloleszczyk,1,Spoiler Master Podcast
1,1243927,tru3,1,Magazyn SFP
2,1418184,Cateborough,1,POPcast - podcast o popkulturze
3,1589044,darek_arest,1,krytyk niezależny
4,3281021,smolinskisebastian,1,krytyk niezależny


In [15]:
df_critics.groupby('priority').count()

,id,userName,publisher
priority,,,
1,9,9,9
2,10,10,10
3,13,13,13
4,17,17,17
5,7,7,7
6,10,10,10
7,1,1,1


In [16]:
df_critics.count()

id           67
userName     67
priority     67
publisher    67
dtype: int64

In [77]:
df_critics.to_csv('intermidate/critics.csv', index=False)

# Dataset preparation
Iterate over critics and get movies which they reviewed

In [144]:
results = []
critics = df_critics['userName'].values.tolist()

In [145]:
it = 0
while len(results) < len(df_critics['userName']):
    for critic in tqdm.tqdm(critics[it:]):
        try:
            movies_ids = list(map(lambda response: {'username': critic, 'movie_id': response[0]}, filmweb.get_reviewed_movies(critic).json()))
            results += movies_ids
        except Exception as e:
            print(f'Exception when getting {critic}\'s reviews. {e}')
            break
        it+=1

100%|██████████| 67/67 [00:10<00:00,  6.37it/s]


In [147]:
df_critics_reviews = pd.DataFrame(results)
df_critics_reviews.head()

,username,movie_id
0,michaloleszczyk,10014103
1,michaloleszczyk,10025735
2,michaloleszczyk,805204
3,michaloleszczyk,877700
4,michaloleszczyk,10026692


In [169]:
for i in tqdm.tqdm(df_critics_reviews.values[2000:]):
    pass

100%|██████████| 4324/4324 [00:00<00:00, 3106040.50it/s]


In [170]:
it = 0
results = []
while len(results) < len(df_critics_reviews.values) or it >= len(df_critics_reviews.values):
    for case in tqdm.tqdm(df_critics_reviews.values[it:]):
        try:
            response = filmweb.get_movie_review(critic_username=case[0], movie_id=case[1]).json()
            if 'comment' in response:
                movie_rating = filmweb.get_movie_rating(case[1]).json()
                results.append({'username': case[0], 
                                'movie_id': case[1],
                                'user_rate': response['rate'],
                                'user_comment': response['comment'],
                                'movie_rating': movie_rating['rate'],
                                'movie_rating_count': movie_rating['count']
                                })
        except Exception as e:
            print(f'Fetching data about movie rating failed: {e}')
            break
        it+=1

  3%|▎         | 165/6324 [03:00<1:52:10,  1.09s/it]


Fetching data about movie rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/tru3/vote/film/10019460 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fc2157464a0>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


  5%|▍         | 295/6159 [04:09<1:22:46,  1.18it/s] 


Fetching data about movie rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/film/164870/rating (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fc215745db0>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


  3%|▎         | 176/5864 [02:45<1:29:21,  1.06it/s]


...

Fetching data about movie rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/MCh092788/vote/film/695409 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fc215744f70>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


 18%|█▊        | 36/195 [02:17<10:06,  3.81s/it]


Fetching data about movie rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/MCh092788/vote/film/31672 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fc2157456c0>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


100%|██████████| 159/159 [00:40<00:00,  3.90it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

KeyboardInterrupt: 

In [171]:
ds_df = pd.DataFrame(results)
ds_df.head()

,username,movie_id,user_rate,user_comment,movie_rating,movie_rating_count
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375


In [172]:
ds_df.count()

username              3057
movie_id              3057
user_rate             3057
user_comment          3057
movie_rating          3057
movie_rating_count    3057
dtype: int64

In [179]:
ds_df.to_json('../dataset/dataset_movies.json')

In [180]:
ds_df.to_csv('../dataset/dataset_movies.csv', index=False, sep='$', encoding='utf8')

TODO: fetch tv series analogously to movies
DO NOT MERGE THE DATASETS! We may consider if we want to train model only on movies, tv series or both
# TV series

In [5]:
df_critics = pd.read_csv('intermidate/critics.csv')
df_critics.head()

,Unnamed: 0,id,userName,priority,publisher
0,0,1164541,michaloleszczyk,1,Spoiler Master Podcast
1,1,1243927,tru3,1,Magazyn SFP
2,2,1418184,Cateborough,1,POPcast - podcast o popkulturze
3,3,1589044,darek_arest,1,krytyk niezależny
4,4,3281021,smolinskisebastian,1,krytyk niezależny


In [6]:
result = []
for critic in tqdm.tqdm(df_critics['userName']):
    response = filmweb.get_reviewed_servies(critic).json()
    result += list(map(lambda review: {'username':critic, 'series_id':review[0]}, response))
df_critics_reviews_series = pd.DataFrame(result)
df_critics_reviews_series.head()

100%|██████████| 67/67 [00:11<00:00,  5.79it/s]


,username,series_id
0,michaloleszczyk,94606
1,michaloleszczyk,867132
2,michaloleszczyk,868514
3,michaloleszczyk,876630
4,michaloleszczyk,862322


In [7]:
it = 0
results = []
while len(results) < len(df_critics_reviews_series.values):
    for case in tqdm.tqdm(df_critics_reviews_series.values[it:]):
        try:
            response = filmweb.get_tv_series_review(critic_username=case[0], series_id=case[1]).json()
            if 'comment' in response:
                series_rating = filmweb.get_movie_rating(case[1]).json()
                results.append({'username': case[0], 
                                'series_id': case[1],
                                'user_rate': response['rate'],
                                'user_comment': response['comment'],
                                'series_rating': series_rating['rate'],
                                'series_rating_count': series_rating['count']
                                })
        except Exception as e:
            print(f'Fetching data about tv series rating failed: {e}')
            break
        it+=1
    if it >= len(df_critics_reviews_series.values):
        break

  7%|▋         | 262/3734 [03:08<41:35,  1.39it/s]


Fetching data about tv series rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/makina13/vote/serial/843886 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f603320fc40>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


  9%|▉         | 329/3472 [04:30<43:02,  1.22it/s]  


Fetching data about tv series rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/ksiawcik_13/vote/serial/809555 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f60329545e0>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


  2%|▏         | 73/3143 [02:25<1:41:54,  1.99s/it]


...

 27%|██▋       | 154/574 [02:45<07:32,  1.08s/it]


Fetching data about tv series rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/mwol/vote/serial/99424 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f6032955990>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


 77%|███████▋  | 322/420 [03:35<01:05,  1.50it/s]


Fetching data about tv series rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/dem3000/vote/serial/654010 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f6032955c60>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


100%|██████████| 98/98 [00:28<00:00,  3.50it/s]


In [8]:
ds_df = pd.DataFrame(results)
ds_df.head()

,username,series_id,user_rate,user_comment,series_rating,series_rating_count
0,michaloleszczyk,867132,8,NH. Grand finale mock-satanistycznej komedii V...,7.57339,436
1,michaloleszczyk,862322,10,Koronkowe komiczne eseje dokumentalne o urokac...,8.27645,4178
2,michaloleszczyk,819951,7,"Znakomicie zrealizowany serial, którego ostatn...",7.85234,10409
3,tru3,10011884,8,Ale to jest dobre! Absolutnie zaskakujący seri...,7.62731,7199
4,tru3,860894,5,Serial pozerski. Główny bohater - Czuły (Borys...,6.07103,4885


In [10]:
ds_df.to_json('../dataset/dataset_tv_series.json')
ds_df.to_csv('../dataset/dataset_tv_series.csv', sep='$', encoding='utf-8')

# Datasets merge

In [51]:
df_movies = pd.read_csv('../dataset/dataset_movies.csv', encoding='utf-8', sep='$')
df_tv_series = pd.read_csv('../dataset/dataset_series2.csv', encoding='utf-8', sep='$')

In [52]:
df_movies = df_movies.rename(axis='columns', mapper={'movie_id': 'show_id', 
                                                     'movie_rating': 'show_rating', 
                                                     'movie_rating_count': 'show_rating_count'})
df_movies.head()

,username,show_id,user_rate,user_comment,show_rating,show_rating_count
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375


In [53]:
df_tv_series = df_tv_series.rename(axis='columns', mapper={'series_id': 'show_id', 
                                                           'series_rating': 'show_rating', 
                                                           'series_rating_count': 'show_rating_count'})
df_tv_series.head()

,username,show_id,user_rate,user_comment,show_rating,show_rating_count
0,michaloleszczyk,867132,8,NH. Grand finale mock-satanistycznej komedii V...,7.57339,436
1,michaloleszczyk,862322,10,Koronkowe komiczne eseje dokumentalne o urokac...,8.27645,4178
2,michaloleszczyk,819951,7,"Znakomicie zrealizowany serial, którego ostatn...",7.85234,10409
3,tru3,10011884,8,Ale to jest dobre! Absolutnie zaskakujący seri...,7.62731,7199
4,tru3,860894,5,Serial pozerski. Główny bohater - Czuły (Borys...,6.07103,4885


In [54]:
df_movies['type'] = 'movie'
df_tv_series['type'] = 'tv_series'

In [55]:
df_dataset = pd.concat([df_movies, df_tv_series], axis=0)
df_dataset

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296,movie
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833,movie
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884,movie
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557,movie
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375,movie
...,...,...,...,...,...,...,...
1326,alekhudzik,849662,3,O wow. Nie udało się zekranizować gry komputer...,7.77990,58245,tv_series
1327,alekhudzik,10012801,6,"No takie ten. ""1 świat 3 świat"", kalejdoskop p...",6.79320,353,tv_series
1328,alekhudzik,875491,7,Dwójka lepsza od jedynki. Zamykamy dyskusję,7.67484,40515,tv_series
1329,alekhudzik,877025,10,"Jest to najlepszy serial ""Coming of age"" świat...",7.22280,579,tv_series


In [57]:
df_dataset = df_dataset.reset_index(drop=True)

In [59]:
df_dataset.to_csv('../dataset/dataset.csv', encoding='utf-8', sep='$', index=False)
df_dataset.to_json('../dataset/dataset.json')

# Dataset cleanup
Filter out empty comments

In [84]:
df_dataset = pd.read_csv('../dataset/dataset.csv', sep='$', encoding='utf-8')
df_dataset

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296,movie
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833,movie
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884,movie
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557,movie
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375,movie
...,...,...,...,...,...,...,...
4383,alekhudzik,849662,3,O wow. Nie udało się zekranizować gry komputer...,7.77990,58245,tv_series
4384,alekhudzik,10012801,6,"No takie ten. ""1 świat 3 świat"", kalejdoskop p...",6.79320,353,tv_series
4385,alekhudzik,875491,7,Dwójka lepsza od jedynki. Zamykamy dyskusję,7.67484,40515,tv_series
4386,alekhudzik,877025,10,"Jest to najlepszy serial ""Coming of age"" świat...",7.22280,579,tv_series


In [85]:
df_dataset = df_dataset[~df_dataset['user_comment'].isna()].reset_index(drop=True)
df_dataset = df_dataset[df_dataset['user_comment'].str.len() > 3].reset_index(drop=True)
df_dataset

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296,movie
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833,movie
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884,movie
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557,movie
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375,movie
...,...,...,...,...,...,...,...
4276,alekhudzik,849662,3,O wow. Nie udało się zekranizować gry komputer...,7.77990,58245,tv_series
4277,alekhudzik,10012801,6,"No takie ten. ""1 świat 3 świat"", kalejdoskop p...",6.79320,353,tv_series
4278,alekhudzik,875491,7,Dwójka lepsza od jedynki. Zamykamy dyskusję,7.67484,40515,tv_series
4279,alekhudzik,877025,10,"Jest to najlepszy serial ""Coming of age"" świat...",7.22280,579,tv_series


In [ ]:
df_dataset.to_csv('../dataset/dataset.csv', encoding='utf-8', sep='$', index=False)
df_dataset.to_json('../dataset/dataset.json')

# Extend dataset with movie/series name

In [43]:
df_dataset = pd.read_csv('../dataset/dataset.csv', encoding='utf-8', sep='$')
df_dataset

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296,movie
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833,movie
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884,movie
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557,movie
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375,movie
...,...,...,...,...,...,...,...
4276,alekhudzik,849662,3,O wow. Nie udało się zekranizować gry komputer...,7.77990,58245,tv_series
4277,alekhudzik,10012801,6,"No takie ten. ""1 świat 3 świat"", kalejdoskop p...",6.79320,353,tv_series
4278,alekhudzik,875491,7,Dwójka lepsza od jedynki. Zamykamy dyskusję,7.67484,40515,tv_series
4279,alekhudzik,877025,10,"Jest to najlepszy serial ""Coming of age"" świat...",7.22280,579,tv_series


In [13]:
def extend_with_title(id):
    response = filmweb.get_title(id)
    if response.status_code == 200:
        if 'originalTitle' not in response.json():
            title = response.json()['title']
        else:
            title = response.json()['originalTitle']
        print(f'{id} -> {response.json()}')
        return (id, title)
    else:
        print(f"Error: {response.status_code} -> {response.content}")
        return (id, None)

In [ ]:
mapped = []

In [28]:
from time import sleep

input = df_dataset['show_id'].to_list()
counter = 173
while counter < len(input):
    print(f"{counter}/{len(input)}")
    try:
        elem = extend_with_title(input[counter])
        counter += 1
        mapped.append(elem)
    except:
        print(f"Failed for: {input[counter]}. Try again in second.")
        sleep(1)
    sleep(0.2)




173/4281
10033068 -> {'title': 'Soulcatcher', 'originalTitle': 'Operacja: Soulcatcher', 'year': 2023, 'type': 'film', 'subType': 'film_cinema', 'posterPath': '/30/68/10033068/8080804.$.jpg'}
174/4281
99390 -> {'title': 'Love Actually Is All Around', 'originalTitle': 'Love Actually', 'year': 2003, 'type': 'film', 'subType': 'film_cinema', 'posterPath': '/93/90/99390/7906420.$.jpg'}
175/4281
10011888 -> {'title': "Mother's Day", 'originalTitle': 'Dzień Matki', 'year': 2023, 'type': 'film', 'subType': 'film_cinema', 'posterPath': '/18/88/10011888/8087448.$.jpg'}
176/4281
857111 -> {'title': 'Swarm', 'originalTitle': 'Rój', 'year': 2023, 'type': 'film', 'subType': 'film_cinema', 'posterPath': '/71/11/857111/8064206.$.jpg'}
177/4281
877030 -> {'title': 'Fanfic', 'originalTitle': 'Fanfik', 'year': 2023, 'type': 'film', 'subType': 'film_cinema', 'posterPath': '/70/30/877030/8068572.$.jpg'}
178/4281
10011964 -> {'title': 'Sick of Myself', 'originalTitle': 'Syk pike', 'year': 2022, 'type': 'fil

In [41]:
columns = ['show_id', 'title']
mapped_df = pd.DataFrame(mapped,columns=columns)
mapped_df

,show_id,title
0,10014103,The Zone of Interest
1,10025735,Passages
2,805204,Killers of the Flower Moon
3,10034333,La passion de Dodin Bouffant
4,10015490,Imaculat
...,...,...
4276,849662,The Last of Us
4277,10012801,Mo
4278,875491,The White Lotus
4279,877025,Reservation Dogs


In [58]:
merge = pd.merge(df_dataset, mapped_df, on='show_id', how='outer').drop_duplicates()
merge.to_csv("../dataset/dataset_with_titles.csv", encoding='utf-8', sep='$', index=False)
merge.to_json("../dataset/dataset_with_titles.json")


# New dataset cases

In [37]:
shows_df = pd.read_csv('../dataset/dataset.csv', encoding='utf-8', sep='$')
shows_df

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296,movie
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833,movie
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884,movie
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557,movie
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375,movie
...,...,...,...,...,...,...,...
4276,alekhudzik,849662,3,O wow. Nie udało się zekranizować gry komputer...,7.77990,58245,tv_series
4277,alekhudzik,10012801,6,"No takie ten. ""1 świat 3 świat"", kalejdoskop p...",6.79320,353,tv_series
4278,alekhudzik,875491,7,Dwójka lepsza od jedynki. Zamykamy dyskusję,7.67484,40515,tv_series
4279,alekhudzik,877025,10,"Jest to najlepszy serial ""Coming of age"" świat...",7.22280,579,tv_series


In [6]:
shows_df = pd.read_csv('../dataset/dataset_with_titles.csv', encoding='utf-8', sep='$')
shows = list(shows_df[['title', 'show_id']].drop_duplicates().to_records(index=False))

In [7]:
shows

[('The Zone of Interest', 10014103),
 ('Passages', 10025735),
 ('Killers of the Flower Moon', 805204),
 ('La passion de Dodin Bouffant', 10034333),
 ('Imaculat', 10015490),
 ('Saw X', 10017756),
 ('Saint Omer', 10016720),
 ('Carmen', 10020792),
 ('El Conde', 10035212),
 ('La donna della domenica', 7043),
 ('A Haunting in Venice', 10021659),
 ('Past Lives', 10029803),
 ('Chien Blanc', 10040235),
 ('Marcel the Shell with Shoes On', 10011754),
 ('Red, White & Royal Blue', 10035218),
 ('No Hard Feelings', 10002968),
 ('Les cinq diables', 10013187),
 ('The Last Voyage of Demeter', 562230),
 ('Gran Turismo', 10014818),
 ('Palm Trees and Power Lines', 10006316),
 ('Reality', 10026113),
 ('Pamfir', 10013416),
 ('Strays', 10019458),
 ('Talk to Me', 10026657),
 ('The Night of the Generals', 37242),
 ('Dialóg 20-40-60', 5040),
 ('The Seven-Per-Cent Solution', 10863),
 ('Thampu', 281473),
 ('Szürkület', 448632),
 ('Longford', 402521),
 ('Oppenheimer', 10002817),
 ('Club Zero', 10032055),
 ('Aranye

In [8]:
def get_range(movie_id):
    response = filmweb.get_movie_info(movie_id).json()
    movie_path = f'{quote(response["title"])}-{response["year"]}-{id}'
    content = filmweb.get_movie_discussion(movie_path).text

    soup = BeautifulSoup(content, 'html.parser')

    pages = soup.select("li.pagination__item:has(a[href])")
    pages = list(map(lambda page: page.find('a', class_='pagination__link').get_text(strip=True), pages))
    def map_to_nums(value):
        try:
            result = int(value)
            return result
        except ValueError:
            return None

    pages = list(map(map_to_nums, pages))
    pages = list(filter(lambda x: x is not None, pages))
    start = 1
    if len(pages) < 2:
        end = start
    else:
        end = max(pages)
    return (start, end, movie_path)


def get_discussion_page(movie_path, page):
    content = filmweb.get_movie_discussion(movie_path, page=page).text
    soup = BeautifulSoup(content, 'html.parser')
    result = []
    matching_divs = soup.select('div.forumTopic')
    only_with_starts_comments = [div for div in matching_divs if div.select_one('.forumTopic__starsNo') is not None]

    for div in only_with_starts_comments:
        forum_text = div.select_one('p.forumTopic__text')
        if forum_text:
            text_content = forum_text.get_text(strip=True)
        else:
            text_content = None

        forum_stars = div.select_one('span.forumTopic__starsNo')
        if forum_text:
            stars_content = int(forum_stars.get_text(strip=True))
        else:
            stars_content = None
        result.append((text_content, stars_content))
    return list(filter(lambda x: x[0] is not None and x[1] is not None, result))
    

In [ ]:
result = []

In [32]:
from time import sleep
counter = 50
while counter < len(shows):
    movie = shows[counter]
    id = movie[1]
    try:
        start, end, movie_path = get_range(id)
        comments = []
        page = start
        while page < end + 1:
            try:
                print(f"Counter: {counter}/{len(shows)}; ID: {id}; page: {page}")
                comments += get_discussion_page(movie_path, page)
                page += 1
            except:
                print("Failed to get comments from the page. Trying again in 1 second...")
                sleep(1)
        
        result += list(map(lambda c: (movie[0], movie[1], c[0], c[1]), comments))
        counter+=1
    except Exception as e:
        print(f"Failed gathering comments from film: {id} {shows[counter][0]}. Trying again in 1 second... {e}")
        sleep(1)
    


Counter: 50/2049; ID: 187; page: 1
Counter: 50/2049; ID: 187; page: 2
Counter: 50/2049; ID: 187; page: 3
Counter: 50/2049; ID: 187; page: 4
Counter: 50/2049; ID: 187; page: 5
Counter: 50/2049; ID: 187; page: 6
Counter: 50/2049; ID: 187; page: 7
Counter: 50/2049; ID: 187; page: 8
Counter: 50/2049; ID: 187; page: 9
Counter: 50/2049; ID: 187; page: 10
Counter: 50/2049; ID: 187; page: 11
Counter: 50/2049; ID: 187; page: 12
Counter: 50/2049; ID: 187; page: 13
Counter: 50/2049; ID: 187; page: 14
Counter: 50/2049; ID: 187; page: 15
Counter: 50/2049; ID: 187; page: 16
Counter: 50/2049; ID: 187; page: 17
Counter: 50/2049; ID: 187; page: 18
Counter: 50/2049; ID: 187; page: 19
Counter: 50/2049; ID: 187; page: 20
Counter: 50/2049; ID: 187; page: 21
Counter: 50/2049; ID: 187; page: 22
Counter: 50/2049; ID: 187; page: 23
Counter: 50/2049; ID: 187; page: 24
Counter: 50/2049; ID: 187; page: 25
Counter: 50/2049; ID: 187; page: 26
Counter: 50/2049; ID: 187; page: 27
Counter: 50/2049; ID: 187; page: 28
C

In [45]:
comments_df = pd.DataFrame(result, columns=['title', 'show_id', 'user_comment', 'user_rate']).drop_duplicates().drop(columns=['title'])
comments_df.to_json('./user_comments.json')
comments_df.to_csv('./user_comments.csv', sep='$', encoding='utf-8', index=False)



In [47]:
dataset_base = shows_df[['show_id', 'show_rating', 'show_rating_count', 'type', 'title']]


In [53]:
from urllib.parse import urlparse

extend = pd.merge(dataset_base, comments_df, on='show_id').drop_duplicates()
extend['username'] = 'anonymous'
extend = extend[extend['user_comment'].str.len() > 15]

def is_url(string):
    try:
        result = urlparse(string)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

url_filter = ~extend['user_comment'].apply(is_url)
extend = extend[url_filter]
extend

,show_id,show_rating,show_rating_count,type,title,user_comment,user_rate,username
0,10014103,8.32095,296,movie,The Zone of Interest,"Film niewątpliwie bardzo mocny, świetne zagran...",9,anonymous
1,10014103,8.32095,296,movie,The Zone of Interest,Czy Strefa interesów naprawdę tak wiele wnosi ...,7,anonymous
2,10014103,8.32095,296,movie,The Zone of Interest,Dwanaście lat temu Lars von Trier wywołał na f...,8,anonymous
3,10014103,8.32095,296,movie,The Zone of Interest,filmie? A weź że się się zdecyduj. Czy niedopo...,7,anonymous
4,10014103,8.32095,296,movie,The Zone of Interest,Żal dupę ściska i robi co w jego mocy. Ale ani...,6,anonymous
...,...,...,...,...,...,...,...,...
323277,698665,7.23548,1911,tv_series,Togetherness,gdyby nie cycki byłoby 4,5,anonymous
323278,270224,8.11299,177,tv_series,Moral Orel,"Serial opowiada o Orelu, który, chcąc być dobr...",6,anonymous
323279,10031643,6.43458,856,tv_series,Waco: American Apocalypse,Dokument wykonany w sposób dość rzetelny. Podo...,7,anonymous
323280,10031643,6.43458,856,tv_series,Waco: American Apocalypse,"Naprawdę dorośli ludzie, którzy im ulegają teg...",5,anonymous


In [54]:
base = pd.read_csv('../dataset/dataset_with_titles.csv', encoding='utf-8', sep='$')
base

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type,title
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296,movie,The Zone of Interest
1,jack_sobczynski,10014103,8,Wielkie kino o odwracaniu oczu od zła. „To nie...,8.32095,296,movie,The Zone of Interest
2,Igi3,10014103,9,Potężny masyw obrazu i dźwięku do rozrywania n...,8.32095,296,movie,The Zone of Interest
3,no_face_no,10014103,9,Niesłychana siła wyparcia przedstawiona w bana...,8.32095,296,movie,The Zone of Interest
4,Maciej_Niedzwiedzki,10014103,9,Coś niespotykanego. Całkowicie nowe i niebanal...,8.32095,296,movie,The Zone of Interest
...,...,...,...,...,...,...,...,...
4276,dem3000,725776,4,"[ep1-6] Schematyczny dramat, który wstępnie pr...",7.36279,7718,tv_series,Humans
4277,dem3000,698665,8,"Każda postać jest tu bardzo ludzka, pełna wad ...",7.23548,1911,tv_series,Togetherness
4278,dem3000,270224,8,"Choć seria jest zabawna i celna, wszystkie te ...",8.11299,177,tv_series,Moral Orel
4279,MCh092788,10039427,5,Bardziej reklama dla potencjalnych nowych właś...,6.82828,99,tv_series,Superpowered: The DC Story


In [55]:
extended_dataset = pd.concat([base, extend], ignore_index=True)
extended_dataset.to_csv('../dataset/extended_dataset.csv', index=False, sep='$', encoding='utf-8')
extended_dataset.to_json('../dataset/extended_dataset.json')

In [61]:
extended_dataset.groupby('title').count().sort_values(by='username', ascending=False)

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type
title,,,,,,,
Avatar,3539,3539,3539,3539,3539,3539,3539
Game of Thrones,3535,3535,3535,3535,3535,3535,3535
The Dark Knight,1733,1733,1733,1733,1733,1733,1733
The Shawshank Redemption,1398,1398,1398,1398,1398,1398,1398
Breaking Bad,1301,1301,1301,1301,1301,1301,1301
...,...,...,...,...,...,...,...
Chien Blanc,1,1,1,1,1,1,1
La vaca que cantó una canción sobre el futuro,1,1,1,1,1,1,1
Til Death Do Us Part,1,1,1,1,1,1,1


In [62]:
extended_dataset[extended_dataset['title'] == 'The Shawshank Redemption']

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type,title
2973,MCh092788,1048,7,Masterpiece feel good movie. Inkluzywne dzieło...,8.79138,914705,movie,The Shawshank Redemption
74896,anonymous,1048,10,zawsze się go dobrze ogląda. Jeden z moich ulu...,8.79138,914705,movie,The Shawshank Redemption
74897,anonymous,1048,10,Mam pytanie. Na końcu filmu pojawia się inform...,8.79138,914705,movie,The Shawshank Redemption
74898,anonymous,1048,6,Według mnie to na pewno pierwsze miejsce powin...,8.79138,914705,movie,The Shawshank Redemption
74899,anonymous,1048,10,Obowiązkowa pozycja każdego kinomana. W tym fi...,8.79138,914705,movie,The Shawshank Redemption
...,...,...,...,...,...,...,...,...
76288,anonymous,1048,10,S.King opowiadanie S.NA SHAWSHANK napisał w ro...,8.79138,914705,movie,The Shawshank Redemption
76289,anonymous,1048,6,"Nie był najgorszy, ale strasznie się wlekł ......",8.79138,914705,movie,The Shawshank Redemption
76290,anonymous,1048,4,Film ok ale bez przesady... zresztą filmwebowy...,8.79138,914705,movie,The Shawshank Redemption
76291,anonymous,1048,5,"Ujdzie na raz, obejrzeć i zapomnieć",8.79138,914705,movie,The Shawshank Redemption
